# Глава 10 Снижение размерности с помощью отбора признаков(МО Крис Элбон)

# Введение

Отбор высококачественных, информативных признаков и удаление менее полезных признаков. Этот подход называется отбором признаков.

3 типа методов отбора признаков:
1)фильтрующие

2)циклические

3)вложенные

Фильтрующие отбирают наилучшие признаки, изучая их статистические свойства.

Циклические методы (wrapper)- для поиска подмножества признаков, которые создают модели с предсказаниями самого высокого качества, используют метод проб и ошибок.

Вложенные методы (embedded)- отбирают наилучшее подмножество признаков как часть или как продолжение процесса тренировки обучающего алгоритма.

# Пороговая обработка дисперсии числовых признаков

Дан набор числовых признаков, требуется удалить те из них, которые имеют низкую дисперсию (содержат мало информации).

Отобрать подмножество признаков с дисперсиями выше заданного порога.

In [1]:
#Загрузить библиотеки
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

In [2]:
#Импортировать немного данных для экспериментирования
iris = datasets.load_iris()

In [3]:
#Создать признаки и цель
features = iris.data
target = iris.target

In [4]:
#Создать обработчик порога 
thresholder = VarianceThreshold(threshold=.5)

In [5]:
#Создать матрицу высокодисперсионных признаков
features_high_variance = thresholder.fit_transform(features)

In [7]:
#Взглянуть на матрицу высокодисперсионных признаков
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

Пороговая обработка дисперсии (variance thesholding, VT) является одним  из основных подходов к отбору признаков. Она мотивированна идеей, что низкодисперсионные признаки скорее всего менее интересны (и полезны), чем высокодисперсионные признаки.

Метод пороговой обработки сначала вычисляет дисперсию каждого признака.

https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Typesetting%20Equations.html

Формула дисперсии: \begin{equation*} Var(x) = 1/n \sum_{i=1}^n (x_i-\mu)^2 \end{equation*}

Далее метод пороговой обработки дисперсии исключает все признаки, исключает все признаки, дисперсия которых не удовлетворяет этому порогу.

Два важных момента:

1)Дисперсия не центрированна, она измеряется в квадратах единиц измерения самого признака. Поэтому этот метод не будет работать, если наборы признаков содержат разные единицы измерения(например, один признак измеряется в годах, другой в долларах). 

2)Порог дисперсии отбирается вручную, поэтому для выбора хорошнго значения мы должны использовать наше собственное суждение.

Дисперсию для каждого признака можно увидеть с помощью атрибута:

variances_

In [9]:
#Взглянуть на дисперсии
thresholder.fit(features).variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

Если признаки были стандартизированы(в нулевое среднее и единичную дисперсию), то пороговая обработка дисперсии будет работать неправильно.

In [11]:
from sklearn.preprocessing import StandardScaler

#стандартизировать матрицу признаков
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

#вычислить дисперсию каждого признака
selector = VarianceThreshold()

selector.fit(features_std).variances_

array([1., 1., 1., 1.])

# Пороговая обработка дисперсии бинарных признаков

https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%91%D0%B5%D1%80%D0%BD%D1%83%D0%BB%D0%BB%D0%B8

Дан набор бинарных, категориальных признаков, и тебуется удалить те из них, которые имеют низкую дисперсию( т е скорее всего содержат мало информации).

Отобрать подмножество признаков  с дисперсией бернулевых случайных величин выше заданного порога.

In [14]:
#Загрузить библиотеки
from sklearn.feature_selection import VarianceThreshold

In [15]:
#создать матрицу признаков
# признак 0 : 80% класс 0
# признак 1 : 80% класс 1
# признак 2 : 60% класс 0, 40% класс - 1

# строки - наблюдения, столбцы - признаки


features = [
    [0, 1, 0],
    [0, 1, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0]
]

In [16]:
# выполнить пороговую обработку дисперсии
thresholder = VarianceThreshold(threshold=(.75 *(1-.75)))

In [17]:
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

#фактически мы отобрали третий столбец, т к в нем дисперсия максимальна (т к в двух других признаках присутствие одного из классов более 75% и они не проходят порог)

Формула расчета дисперсии для бинарных признаков:

Var(x) = p * (1-p)

# Обработка высококоррелированных признаков

Дана матрица признаков, и есть подозрения, что некоторые признаки сильно коррелированы.

Использовать корреляционную матрицу для выполнения проверки на сильно коррелированные признаки. Если существуеют сильно коррелированные признаки, то рассмотреть возможность исключения одного из коррелированных признаков.

In [1]:
#загрузить библиотеки
import pandas as pd
import numpy as np

In [2]:
#создать матрицу признаков с высококоррелированными признаками
features = np.array([
    [1, 1, 1],
    [2, 2, 0],
    [3, 3, 1],
    [4, 4, 0],
    [5, 5, 1],
    [6, 6, 0],
    [7, 7, 1],
    [8, 7, 0],
    [9, 7, 1]
])

In [3]:
#конвертировать матрицу признаков во фрейм данных
dataframe = pd.DataFrame(features)

In [5]:
#создать корреляционную матрицу
corr_matrix = dataframe.corr().abs()
corr_matrix

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,0.034503
2,0.000000,0.034503,1.000000


In [9]:
ones = np.ones(corr_matrix.shape)
ones

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [10]:
trius = np.triu(ones, k=1)
trius

array([[0., 1., 1.],
       [0., 0., 1.],
       [0., 0., 0.]])

In [11]:
trius_bool = trius.astype(np.bool)
trius_bool

array([[False,  True,  True],
       [False, False,  True],
       [False, False, False]])

In [12]:
#выбрать верхний треугольник корреляционной матрицы
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [13]:
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


In [16]:
#найти индекс столбцов признаков с корреляцией больше 0.95
to_drop = [column for column in upper.columns if any (upper[column] > 0.95)]
to_drop

[1]

In [ ]:
#с индексом 1 - колонку нужно убрать

In [17]:
#исключить признаки
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In [18]:
dataframe.drop(dataframe.columns[to_drop], axis=1)

,0,2
0,1,1
1,2,0
2,3,1
3,4,0
4,5,1
5,6,0
6,7,1
7,8,0
8,9,1


Коррелированные признаки - одна из проблем. Если два признака сильно коррелированы, то информация, которую они содержат - очень похожа. Включать оба признака - будет лишним. Решение: удалить один из них из набора признаков. 

1)Создаем корреляционную матрицу всех признаков

In [19]:
dataframe.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


2)Смотрим на верхний треугольник корреляционной матрицы, чтобы определить пары сильно коррелированных признаков

In [21]:
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


Из каждой пары из набора признаков - мы удаляем по одному признаку.

# Удаление нерелевантных признаков для классификации

Дан категориальный вектор целей, и требуется удалить неинформативные признаки

Если признаки являются категориальными, то вычислить статистический показатель хи-квадрат между каждым признаком и вектором целей

https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%85%D0%B8-%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82

https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D0%B9_%D1%81%D0%BE%D0%B3%D0%BB%D0%B0%D1%81%D0%B8%D1%8F_%D0%9F%D0%B8%D1%80%D1%81%D0%BE%D0%BD%D0%B0

https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%B7%D0%BD%D0%B0%D1%87%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D1%8C

In [7]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

In [8]:
#загрузить данные
iris = load_iris()
features = iris.data
target = iris.target

In [9]:
features.shape

(150, 4)

In [10]:
features[:10, :]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [11]:
#конвертировать в категориальные данные путем преобразования данных в целые числа
features = features.astype(int)

In [12]:
features[:10, :]

array([[5, 3, 1, 0],
       [4, 3, 1, 0],
       [4, 3, 1, 0],
       [4, 3, 1, 0],
       [5, 3, 1, 0],
       [5, 3, 1, 0],
       [4, 3, 1, 0],
       [5, 3, 1, 0],
       [4, 2, 1, 0],
       [4, 3, 1, 0]])

In [15]:
#Отобрать два признака с наивысшими значениями статистического показателя хи-квадрат 
#(наиболее влиятельные признаки)
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

In [17]:
print("Исходное количество признаков: ", features.shape[1])
print("Сокращенное количество признаков: ", features_kbest.shape[1])

Исходное количество признаков:  4
Сокращенное количество признаков:  2


Если признаки являются количественными, то вычислить статистический показатель F дисперсионного анализа, между каждым признаком и вектором целей.

Отобрать два признака  с наивысшими значениями статистического показателя F

In [18]:
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

In [19]:
print("Исходное количество признаков: ", features.shape[1])
print("Сокращенное количество признаков: ", features_kbest.shape[1])

Исходное количество признаков:  4
Сокращенное количество признаков:  2


In [20]:
from sklearn.feature_selection import SelectPercentile
#отобрать 75% признаков  с наивысшими значениями статистического показателя F
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

In [21]:
print("Исходное количество признаков: ", features.shape[1])
print("Сокращенное количество признаков: ", features_kbest.shape[1])

Исходное количество признаков:  4
Сокращенное количество признаков:  3


Статистический показатель хи-квадрат проверяет независимость двух категориальных векторов.

Этот статистический показатель является разницей между наблюдаемым числом наблюдений в каждом классе категориального признака, и тем,  что мы ожидали бы, если бы этот объект был независимым от вектора целей( т е не связанным с ним).

\begin{equation*} \chi^2 =  \sum_{i=1}^n (O_i-E_i)^2/E_i \end{equation*}

\begin{equation*}O_i\end{equation*}  - количество  наблюдений в классе i

\begin{equation*}E_i\end{equation*}  - количество  наблюдений в классе i, которое мы ожидаем, если нет связи, между объектом и вектором целей

Статистический показатель хи-квадрат представляет собой единственное число, которое говорит о том, насколько велика разница между вашими наблюдаемыми количествами и количествами, которые вы ожидали бы, если бы в популяции не было никакой связи вообще.

Вычисляя статистический показатель хи-квадрат между признаком и вектором целей, мы получаем меру независимости между ними.
Если вектор целей не зависит от признаковой переменной, то она не годится для нас, поскольку не содержит информации, которую мы можем использовать для классификации.

С другой стороны, если две величины сильно зависят, то они, скорее всего, очень информативны для тренировки нашей модели.

Для того, чтобы использовать хи-квадрат в отборе признаков, мы вычисляем статистический показатель хи-квадрат между каждым признаком и вектором целей, а затем отбираем признаки с наилучшими статистическим показателями хи-квадрат.

В библиотеке scikit-learn можно использовать SelectKBest для отбора признаков с наилучшими статистическими показателями. Параметр к - количество признаков, которые мы хотим оставить.

Нужно: чтобы и вектор целей и признаки были категориальными. (Или нужно преобразовать.)
    Все значения должны быть неотрицательными.

Альтернатива(для числовых признаков):

Использовать метод f_classif - для вычисления статистического показателя F дисперсионного анализа с каждым признаком и вектором целей.

Оценочные значения F проверяют - различаются ли значимо средние значения для каждой группы, когда мы группируем этот числовой признак по вектору целей.

Например: бинарный признак пол и количественный признак - экзаменнационные баллы, то F (оценночный признак) говорил бы нам, отличается ли средний экзаменнационный бал для мужчин от среднего экзаменнационного балла для женщин.

# Рекурсивное устранение признаков

Требуется возможность автоматически отбирать наилучшие признаки для использования в дальнейшем

Для выполнения рекурсивного устранения признаков RFE (recursive feature elimination) использовать класс RFECV библиотеки scikit-learn  совместно с перекрестной проверкой CV (cross-validation). То есть многократно тренировать модель, каждый раз удаляя признак до тех пор, пока пока результативность модели (например, точность) не станет хуже. Оставшиеся признаки являются наилучшими:

In [1]:
import warnings
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

In [2]:
#убрать раздражающее, но безвредное предупреждение
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [3]:
#сгенирировать матрицу признаков, вектор целей и истинные коэффициенты
features, target = make_regression(n_samples=10000,
                                  n_features=100,
                                  n_informative=2,
                                  random_state=1)

In [6]:
#создать объект линейной регрессии
ols = linear_model.LinearRegression()

In [8]:
#рекурсивно устранить признаки
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
new_features = rfecv.transform(features)
new_features

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


array([[ 0.00850799,  0.7031277 ,  0.57847319, -0.09527592, -0.34606121],
       [-1.07500204,  2.56148527, -0.2823868 ,  0.93276803, -1.8392567 ],
       [ 1.37940721, -1.77039484, -0.07206395, -0.43567608, -0.90016708],
       ...,
       [-0.80331656, -1.60648007,  1.05461415,  0.58281391, -1.28329706],
       [ 0.39508844, -1.34564911, -0.21056771,  0.82308453,  0.85012142],
       [-0.55383035,  0.82880112,  1.52184243,  0.91532556,  0.27741159]])

In [9]:
print("Исходное количество признаков: ", features.shape[1])
print("Сокращенное количество признаков: ", new_features.shape[1])

Исходное количество признаков:  100
Сокращенное количество признаков:  5


In [10]:
#после того, как мы выполнили рекурсивное устранение признаков, 
#мы можем увидеть количество признаков, которые мы должны оставить
rfecv.n_features_

5

Мы также можем увидеть, какие именно из этих признаков мы должны оставить

In [11]:
#какие категории самые лучшие
rfecv.support_

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

Мы даже можем взгрянуть на ранги признаков.

In [12]:
#ранжировать признаки от самого лучшего до самого плохого
rfecv.ranking_

array([48, 45, 64, 72, 26,  1, 31, 34, 37, 33, 39, 43,  3, 30, 28, 49, 40,
       86,  8, 25, 80, 82, 52, 93, 22, 96, 71, 18, 11, 57, 38, 74, 24,  7,
       15, 19, 63, 73, 55,  1, 12, 36, 62, 94, 17, 76,  1, 35, 88, 87, 91,
       10, 23, 56, 27, 95, 20,  5, 81, 46,  2, 13, 75,  4, 44, 65, 85, 78,
       51, 47, 84, 58, 66, 60, 77, 32,  1, 29,  1, 67, 42, 70,  6, 69, 83,
       79,  9, 53, 92, 54, 50, 16, 41, 59, 14, 68, 89, 21, 90, 61])

Идея рекурсивного устранения признаков RFE заключается в неоднократной тренировки модели, которая содержит некоторые параметры (веса и коэффициенты). Модели - такой как линейная регрессия и опорно-векторные машины.

Дополнительно: Метод рекурсивного исключения признаков (recursive feature elimination, RFE) реализует следующий алгоритм: модель обучается на исходном наборе признаков и оценивает их значимость, затем исключается один или несколько наименее значимых признаков, модель обучается на оставшихся признаках, и так далее, пока не останется заданное количество лучших признаков. 

Первый раз, когда мы тренируем модель, мы включаем все признаки. Затем мы находим признак с наименьшим параметром(мы исходим из того, что признаки прошкалированы и стандартизированы), т е удаляем наименее важный признак и удаляем его из набора признаков.

Сколько признаков мы должны оставить? Мы можем повторять этот цикл, пока у нас не останется всего один признак.

Более верный подход требует, чтобы мы включили новую концепцию - перекрестная проверка CV (cross-validation)

Идея(СV): при наличии данных, содержащих цель, которую мы хотим предсказать и матрицу признаков,
    
во-первых: мы разделяем данные на 2 группы - тренировочный набор и тестовый,
    
во-вторых: мы тренируем модель, используя тренировочный набор,
    
в-третьих: мы делаем вид, что не знаем цели тестового набора, и применяем модель к признакам тестового набора, чтобы предсказать значения тестового набора.
    
в-четвертых: мы мы сравниваем наши предсказанные целевые значения с истинными целевыми значениями, чтобы оценить нашу модель.

Перекрестная проверка может использоваться для поиска оптимального количества признаков, которые следует оставить во время рекурсивного устранения признаков.

В рекурсивном устранении признаков с перекрестной проверкой после каждой итерации мы используем перекрестную проверку для оценивания нашей модели.

Если перекрестная проверка показывает, что после того, как мы исключили признак, наша модель улучшилась, мы продолжаем следующий цикл.

Однако, если она показывает, что после того, как мы исключили признак, наша модель ухудшилась, мы помещаем этот признак обратно в набор признаков и отбираем эти признаки, как наилучшие.

В sklearn рекурсивное устранение признаков с перекрестной проверкой реализуется с помощью класса RFECV, который содержит параметры:

estimator - определяет тип модели, которую мы хотим натренировать (например, линейную регрессию)

step - задает количество или долю признаков, которые необходимо удалять во время каждого цикла

scoring - задает метрический показатель качества, который мы используем для оценивания нашей модели во время перекрестной проверки


